## Lab 3: Introduction to AWS Wrangler

#### Load Credentials

In [1]:
# load your credentials
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# import awswrangler
import awswrangler as wr

#### Making sure it works

In [3]:
# test if you can read from a private bucket

df = wr.s3.read_csv('s3://techcatalyst-raw/stocks/GOOG.csv')
df.head()

,Date,Open,High,Low,Close,Volume
0,1/2/2025 16:00:00,191.49,193.20,188.71,190.63,17545162
1,1/3/2025 16:00:00,192.73,194.50,191.35,193.13,12874957
2,1/6/2025 16:00:00,195.15,199.56,195.06,197.96,19483323
3,1/7/2025 16:00:00,198.27,202.14,195.94,196.71,16966760
4,1/8/2025 16:00:00,193.95,197.64,193.75,195.39,14335341


In [4]:
# the returned object is actually a pandas DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    140 non-null    object 
 1   Open    140 non-null    float64
 2   High    140 non-null    float64
 3   Low     140 non-null    float64
 4   Close   140 non-null    float64
 5   Volume  140 non-null    int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 6.7+ KB


#### Inspect Glue databases

In [5]:
databases = wr.catalog.databases()
print(databases)

               Database       Description
0             aamnah_db                  
1           aamnah_taxi                  
2             alexia_db                  
3           alexia_logs                  
4           alexia_song                  
5      awswrangler_test                  
6                ben_db                  
7              ben_song                  
8              ben_taxi                  
9            blake_taxi                  
10          blake_wr_db                  
11              db_name                  
12              default  default database
13              emma_db                  
14            emma_taxi                  
15           fabiola_db                  
16         fabiola_taxi                  
17           jaden_taxi                  
18        jadenastle_db                  
19           melissa_db                  
20         melissa_logs                  
21        melissa_songs                  
22         melissa_taxi           

#### Creating a database

In [7]:
name = 'fabiola'
database_name = f"{name}_db"
#wr.catalog.create_database(database_name)

In [8]:
# confirm new database
databases = wr.catalog.databases()
print(databases)

               Database       Description
0             aamnah_db                  
1           aamnah_taxi                  
2             alexia_db                  
3           alexia_logs                  
4           alexia_song                  
5      awswrangler_test                  
6                ben_db                  
7              ben_song                  
8              ben_taxi                  
9            blake_taxi                  
10          blake_wr_db                  
11              db_name                  
12              default  default database
13              emma_db                  
14            emma_taxi                  
15           fabiola_db                  
16         fabiola_taxi                  
17           jaden_taxi                  
18        jadenastle_db                  
19           melissa_db                  
20         melissa_logs                  
21        melissa_songs                  
22         melissa_taxi           

In [9]:
# check tables in new database
wr.catalog.tables(database=database_name)

,Database,Table,Description,TableType,Columns,Partitions
0,fabiola_db,fabiola_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",


#### Writing to database

In [10]:
# write parquet file to database
wr.s3.to_parquet(
    df=df, # the DataFrame you just created 
    path=f"s3://techcatalyst-raw/{name}/", # write to the techcatalyst-raw bucket under your folder name (or it would create a new folder if it does not exist)
    dataset=True, 
    database=database_name, # the name of the database you just created in AWS Glue 
    table= 'fabiola_stock', # pick a table name for example YOURNAME_STOCK
    mode='overwrite'
    )

{'paths': ['s3://techcatalyst-raw/fabiola/cb6ef340d3e642efa80e014a49abe2d4.snappy.parquet'],
 'partitions_values': {}}

In [11]:
# verify
wr.catalog.tables(name_contains="stock")

,Database,Table,Description,TableType,Columns,Partitions
0,aamnah_db,aamnah_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
1,alexia_db,alexia_stock,,EXTERNAL_TABLE,"date, open, high, low, close, volume",
2,ben_db,ben_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
3,blake_wr_db,blake_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
4,emma_db,emma_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
5,fabiola_db,fabiola_stock,,EXTERNAL_TABLE,"date, open, high, low, close, volume",
6,jadenastle_db,jaden_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
7,melissa_db,melissa_stocks,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
8,michael_db,michael_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",
9,miraj_db,miraj_google_stock,This is my stock table.,EXTERNAL_TABLE,"date, open, high, low, close, volume",


#### Read table as df from database

In [12]:
df = wr.s3.read_parquet_table(database=database_name, 
                              table='fabiola_stock')

# Display the DataFrame's first few rows
df.head()

,date,open,high,low,close,volume
0,1/2/2025 16:00:00,191.49,193.20,188.71,190.63,17545162
1,1/3/2025 16:00:00,192.73,194.50,191.35,193.13,12874957
2,1/6/2025 16:00:00,195.15,199.56,195.06,197.96,19483323
3,1/7/2025 16:00:00,198.27,202.14,195.94,196.71,16966760
4,1/8/2025 16:00:00,193.95,197.64,193.75,195.39,14335341


In [13]:
# column types
wr.catalog.get_table_types(database=database_name, 
                           table='fabiola_stock')

{'date': 'string',
 'open': 'double',
 'high': 'double',
 'low': 'double',
 'close': 'double',
 'volume': 'bigint'}

In [14]:
# glue metadata
table_details = wr.catalog.get_tables(database=database_name)

next(table_details)

{'Name': 'fabiola_stock',
 'DatabaseName': 'fabiola_db',
 'CreateTime': datetime.datetime(2025, 8, 4, 19, 50, 49, tzinfo=tzlocal()),
 'UpdateTime': datetime.datetime(2025, 8, 5, 12, 23, 55, tzinfo=tzlocal()),
 'Retention': 0,
 'StorageDescriptor': {'Columns': [{'Name': 'date', 'Type': 'string'},
   {'Name': 'open', 'Type': 'double'},
   {'Name': 'high', 'Type': 'double'},
   {'Name': 'low', 'Type': 'double'},
   {'Name': 'close', 'Type': 'double'},
   {'Name': 'volume', 'Type': 'bigint'}],
  'Location': 's3://techcatalyst-raw/fabiola/',
  'InputFormat': 'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat',
  'OutputFormat': 'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat',
  'Compressed': True,
  'NumberOfBuckets': -1,
  'SerdeInfo': {'SerializationLibrary': 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe',
   'Parameters': {'serialization.format': '1'}},
  'BucketColumns': [],
  'SortColumns': [],
  'Parameters': {'CrawlerSchemaDeserializerVersi

In [15]:
# Example adding additional metadata information 

desc = "This is my stock table."
param = {"source": "Google", "class": "stock"}
comments = {
    "Date": "Trading Date",
    "Open": "Opening Price",
    "Close": "Closing Price"
}

wr.s3.to_parquet(
    df=df,
    path=f"s3://techcatalyst-raw/{name}/",
    dataset=True,
    database=database_name,
    table='fabiola_stock',
    mode='overwrite',
    glue_table_settings=wr.typing.GlueTableSettings(description=desc,  # here we are passing some metadata
                                                    parameters=param, 
                                                    columns_comments=comments),
    )

{'paths': ['s3://techcatalyst-raw/fabiola/4ce04d7cf0854cd9a18baec4f5baa980.snappy.parquet'],
 'partitions_values': {}}

In [16]:
wr.catalog.table(database=database_name, table='fabiola_stock')

,Column Name,Type,Partition,Comment
0,date,string,False,Trading Date
1,open,double,False,Opening Price
2,high,double,False,
3,low,double,False,
4,close,double,False,Closing Price
5,volume,bigint,False,


#### List objects in bucket

In [17]:
wr.s3.list_objects('s3://techcatalyst-raw/stage/')

['s3://techcatalyst-raw/stage/yellow_tripdata.csv',
 's3://techcatalyst-raw/stage/yellow_tripdata.json',
 's3://techcatalyst-raw/stage/yellow_tripdata.parquet']

#### Download file from S3

In [18]:
wr.s3.download(path='s3://techcatalyst-raw/stocks/GOOG.csv', 
               local_file='wr_GOOG.csv')

#### Upload file to S3

In [19]:
your_name = 'fabiola'
file_name = 'wr_GOOG.csv'
wr.s3.upload(local_file='wr_GOOG.csv',path= f's3://techcatalyst-raw/{your_name}/uploads/{file_name}')

In [20]:
wr.s3.list_objects(f's3://techcatalyst-raw/{your_name}/uploads/')

['s3://techcatalyst-raw/fabiola/uploads/wr_GOOG.csv']

### Exercise (using Glue Catalog and Athena)

#### Define Configuration

In [21]:
# A name for the new Glue Database it should be YOURNAME_TAXI
db_name = 'fabiola_taxi'

# A name for the table it should be YOURNAME_TRIPDATA
table_name = 'fabiola_tripdata'

# the path_direcoty to should point to the bucket (main directory)
s3_path_directory = 's3://techcatalyst-raw/taxi_data/'

# The path_file should be the full path to the actual file
s3_path_file = 's3://techcatalyst-raw/taxi_data/yellow_tripdata_2024-01.parquet'


#### Get the Schema from the Parquet File Metadata

In [22]:
# uncomment below if you ran into issues to clean things up and rerun the cell
wr.catalog.delete_table_if_exists(database=db_name, table=table_name) 

# Create the new Glue database first based on the db_name you created
#wr.catalog.create_database(db_name)

# This function can extract the schema from our file and returns a tuple: (schema, partitions). We only need the schema. 
columns_types, partitions_types = wr.s3.read_parquet_metadata(path=s3_path_file)
print("Successfully read schema from Parquet file.")

Successfully read schema from Parquet file.


#### Create the Glue Table with the Explicit Schema

In [23]:

wr.catalog.create_parquet_table(
    database=db_name, # pass the database name
    table=table_name, # pass the table name
    path=s3_path_directory, # use the directoy here 
    columns_types=columns_types,  # Pass the schema here
    partitions_types=partitions_types
)
print(f"Table '{table_name}' created successfully in database '{db_name}'.")

Table 'fabiola_tripdata' created successfully in database 'fabiola_taxi'.


In [24]:
query = f"SELECT * FROM {table_name} LIMIT 5"

df = wr.athena.read_sql_query(query, database=db_name)

print("\nQuery Results:")
print(df)


Query Results:
   vendorid tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2024-04-03 17:49:47   2024-04-03 18:39:29              NaN   
1         2  2024-04-03 17:01:56   2024-04-03 17:31:43              NaN   
2         2  2024-04-03 17:42:09   2024-04-03 17:42:57              NaN   
3         2  2024-04-03 17:58:05   2024-04-03 18:04:41              NaN   
4         2  2024-04-03 17:27:15   2024-04-03 17:58:46              NaN   

   trip_distance  ratecodeid store_and_fwd_flag  pulocationid  dolocationid  \
0           3.77         NaN               <NA>           236           234   
1           3.32         NaN               <NA>            87           186   
2           0.00         NaN               <NA>           246           246   
3           0.70         NaN               <NA>           233           233   
4           4.63         NaN               <NA>           237           246   

   payment_type  fare_amount  extra  mta_tax  tip_amount  